In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests # Peticiones HTTP
from bs4 import BeautifulSoup # Web Scrapping
import time # Prevenir detección de Scrapping
import traceback # Visualización de excepciones
from selenium import webdriver
import random
import os

In [108]:
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

**IMPORTANTE:** El link no va bien porque te pone la posicion de donde estes aunque se lo especifiques, asi que lo suyo sería una vez abierto el driver ir a la web y hacer la busqueda nosotros.

In [75]:
# Web con los datos
url = 'https://www.micole.net/panel/busqueda-avanzada?latitude=40.41955&longitude=-3.69196&scope_id=1|3|2&get_maps=1&range=15'

driver.get(url)
time.sleep(5)

In [61]:
# Cargamos el html en un objeto BeautifulSoap
html = BeautifulSoup(driver.page_source)
# Cargamos todas los colegios
div_list = html.find_all('div',{'class':'advSearchResults'})

# Name
name = div_list[0].find_all('a')[1].text

# Address
address = div_list[0].find('small').text

# Price
price = div_list[0].find('div',{'class':'my-2 d-none d-md-block'}).find('h3').get('title')

# Review
review = div_list[0].find('div',{'class':'ratings'}).find('span').get('aria-label')

# Description
description = div_list[0].find('div',{'class':'my-2'}).find('h3').text
print(name, address, price, review, description, sep='\n')

International School Of Madrid
Calle De Rosa Jardón 1, 28016, Madrid, Madrid
Más de 700€/mes
4.7 estrellas
Colegio Privado de Educación Infantil, Primaria, Secundaria y Bachillerato


In [63]:
# Funcion para extraer la información
def extract_info(div):
    return {
        'name': div.find_all('a')[1].text,
        'address': div.find('small').text,
        'price': div.find('div',{'class':'my-2 d-none d-md-block'}).find('h3').get('title'),
        'review': div.find('div',{'class':'ratings'}).find('span').get('aria-label'),
        'description': div.find('div',{'class':'my-2'}).find('h3').text
    }

info_list = [extract_info(div) for div in div_list]
info_list[0:2]

[{'name': 'International School Of Madrid',
  'address': 'Calle De Rosa Jardón 1, 28016, Madrid, Madrid',
  'price': 'Más de 700€/mes',
  'review': '4.7 estrellas',
  'description': 'Colegio Privado de Educación Infantil, Primaria, Secundaria y Bachillerato'},
 {'name': 'Los Tilos',
  'address': 'Calle el Bosco s/n, 28038, Madrid, Madrid',
  'price': 'Entre 100€ y 300€/mes',
  'review': '3.8 estrellas',
  'description': 'Colegio Concertado de Educación Infantil, Primaria, Secundaria y Bachillerato'},
 {'name': 'Highlands School El Encinar',
  'address': 'Calle de San Enrique de Ossó 46, 28050, Madrid, Madrid',
  'price': 'Más de 700€/mes',
  'review': '3.3 estrellas',
  'description': 'Colegio Privado de Educación Infantil, Primaria, Secundaria y Bachillerato'},
 {'name': 'Alameda De Osuna',
  'address': 'Paseo de la Alameda de Osuna 60, 28042, Madrid, Madrid',
  'price': 'Entre 300€ y 700€/mes',
  'review': '3.5 estrellas',
  'description': 'Colegio Privado de Educación Infantil, Prim

**IMPORTANTE:** La página tiene scroll infinito, por lo que para cargar todos los colegios tenemos que ir haciendo Scroll hasta que se carguen todos. Para saber cuando estan cargados simplemente tenemos que fijarnos en que saldra el icono de que esta cargando más pero no hará nada. Debido a que un exceso de colegios puede reventar el driver, lo suyo es no cargar más de 1000 colegios.  Para este caso he ido ejecuntado las siguientes celdas para cada uno de los tipos (Concerta, Privado y Publico) para evitar que reventase el driver. El siguiente código no sabe cuando se ha acabado de cargar por ello vamos incrementando o reduciendo el rango del loop hasta que validemos **nosotros** que estan todos cargados. 

In [112]:
# Código para scrollear hasta tenerlos todos cargados
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec

for i in range(0, 25):
    driver.execute_script("arguments[0].scrollIntoView({block: \"center\", inline: \"center\"});", driver.find_element_by_xpath(f"//footer"))
    time.sleep(scroll_pause_time)

In [113]:
html = BeautifulSoup(driver.page_source)
div_list = html.find_all('div',{'class':'advSearchResults'})
info_list = [extract_info(div) for div in div_list]
df = pd.DataFrame(info_list)
df.head()

,name,address,price,review,description
0,International School Of Madrid,"Calle De Rosa Jardón 1, 28016, Madrid, Madrid",Más de 700€/mes,4.7 estrellas,"Colegio Privado de Educación Infantil, Primari..."
1,Highlands School El Encinar,"Calle de San Enrique de Ossó 46, 28050, Madrid...",Más de 700€/mes,3.3 estrellas,"Colegio Privado de Educación Infantil, Primari..."
2,Alameda De Osuna,"Paseo de la Alameda de Osuna 60, 28042, Madrid...",Entre 300€ y 700€/mes,3.5 estrellas,"Colegio Privado de Educación Infantil, Primari..."
3,Arcángel Rafael,"Calle de Maqueda 4, 28024, Madrid, Madrid",Entre 300€ y 700€/mes,4.3 estrellas,"Colegio Privado de Educación Infantil, Primari..."
4,Everest,"Urb. Monteclaro, 28223, Pozuelo de Alarcón, Ma...",Entre 300€ y 700€/mes,4.2 estrellas,"Colegio Privado de Educación Infantil, Primari..."


In [114]:
# Aqui se hace esto por lo comentado anteriormente
df["type"] = "Privado"

1786

In [115]:
df_joined = df_joined.append(df.drop_duplicates())
len(df_joined)

1786

In [116]:
df_joined.type.unique()

array(['Publico', 'Concertado', 'Privado'], dtype=object)

In [117]:
df_joined.to_csv("./colegios_madrid.csv")